In [21]:
import pandas as pd 
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
import requests
from config_api import weather_apiKey as wkey

In [22]:

# Create list for weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + wkey

Beginning Data Retrieval     
-----------------------------


In [23]:
# Create 2000 random latitude and longitude coordinates 
lats = np.random.uniform(-90, 90, size = 2000)
lngs = np.random.uniform(-180, 180, size = 2000)
lats_lngs = zip(lats, lngs)
lats_lngs

# Add coordinates to a list 
latlngs_lst = list(lats_lngs)

In [24]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for latlng in latlngs_lst:
    city = citipy.nearest_city(latlng[0],latlng[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    # print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)

724

In [25]:

# Loop through city Lists 
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 
    if (i % 50 == 0 and i >= 50):
        set_count += 1 
        record_count = 1 
    
    # Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Run an API request for each of the cities
    try:
       
        # Parse the JSON data
        city_weather = requests.get(city_url).json()

        # Compile needed data
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        

        # Fill City Data list 
        city_data.append({"City": city.title(),
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         })
        
    #If an error occurs
    except: 
        print("--- City not found")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | margate
Processing Record 2 of Set 1 | cabo san lucas
Processing Record 3 of Set 1 | punta arenas
Processing Record 4 of Set 1 | saint-philippe
Processing Record 5 of Set 1 | lebu
Processing Record 6 of Set 1 | mataura
Processing Record 7 of Set 1 | atuona
Processing Record 8 of Set 1 | haibowan
--- City not found
Processing Record 9 of Set 1 | pangody
Processing Record 10 of Set 1 | oga
Processing Record 11 of Set 1 | chokurdakh
Processing Record 12 of Set 1 | leningradskiy
Processing Record 13 of Set 1 | ushuaia
Processing Record 14 of Set 1 | avarua
Processing Record 15 of Set 1 | hamilton
Processing Record 16 of Set 1 | ixtapa
Processing Record 17 of Set 1 | mosalsk
Processing Record 18 of Set 1 | carnarvon
Processing Record 19 of Set 1 | samusu
--- City not found
Processing Record 20 of Set 1 | port alfred
Processing Record 21 of Set 1 | lagoa
Processing Record 22 of Set 1 | skjervoy
Processing Record 23 of Set 1 | samarai
Processing Record 24 of Set

In [26]:
len(city_data)

667

In [28]:

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Margate,GB,2021-07-22 13:53:50,51.3813,1.3862,79.88,70,0,1.99
1,Cabo San Lucas,MX,2021-07-22 14:00:25,22.8909,-109.9124,83.52,66,1,5.66
2,Punta Arenas,CL,2021-07-22 14:00:26,-53.1500,-70.9167,35.11,93,100,6.69
3,Saint-Philippe,RE,2021-07-22 14:00:26,-21.3585,55.7679,72.99,74,50,12.95
4,Lebu,CL,2021-07-22 14:00:26,-37.6167,-73.6500,50.09,83,99,3.53
5,Mataura,NZ,2021-07-22 14:00:26,-46.1927,168.8643,40.48,97,10,2.26
6,Atuona,PF,2021-07-22 14:00:26,-9.8000,-139.0333,76.77,77,100,12.57
7,Pangody,RU,2021-07-22 13:56:03,65.8500,74.4859,64.58,54,30,13.40
8,Oga,IT,2021-07-22 14:00:27,46.4631,10.3470,72.14,62,27,2.80
9,Chokurdakh,RU,2021-07-22 14:00:27,70.6333,147.9167,38.59,84,58,8.39


In [31]:
# Create the output file 
output_data_file = 'Weather Database/cities.csv'

# Export the City_Data into a CSV
city_data_df.to_csv(output_data_file, index_label='City_ID')